In [1]:
import os
os.environ["SSL_CERT_FILE"] = "/mnt/d/Travel Assistant/Musafir/Fortinet_CA_SSL(15).cer"
os.environ["REQUESTS_CA_BUNDLE"] = "/mnt/d/Travel Assistant/Musafir/Fortinet_CA_SSL(15).cer"

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("../data/processed_data/all_cities_combined_clean.csv")

In [4]:
df

,city,section,subsection,text
0,Cairo,See,Further afield,29.8 31.233333 1 Dahshur Pyramids . For a cont...
1,Cairo,See,NaN,30.0405 31.2646 2 Al-Azhar Park . A landscaped...
2,Cairo,See,NaN,30.0417 31.2483 4 Abdeen Palace . About 1 km e...
3,Cairo,See,NaN,30.0475 31.262222 3 Khan El Khalily . Cairo's ...
4,Cairo,See,NaN,30.047778 31.233333 1 Egyptian Museum ( 250 m ...
...,...,...,...,...
538,Seoul,Districts,NaN,"West ( Gangseo-gu, Yangcheon-gu ) Western area..."
539,Seoul,See,NaN,While Seoul today is mostly known as a super-m...
540,Seoul,Districts,NaN,Yeongdeungpo & Guro (영등포 / 구로) ( Yeongdeungpo-...
541,Seoul,Districts,NaN,Yongsan (용산) ( Yongsan-gu ) Yongsan is home to...


In [5]:
# change csv to json 
documents = df.to_dict(orient='records')

In [6]:
documents[495:500]

[{'city': 'Seoul',
  'section': 'Stay safe',
  'subsection': nan,
  'text': 'Fake monks have been known to operate in Seoul, notably around the Jogyesa temple. They are dressed as Buddhist monks requesting donations from people on the street in return for blessings, although they do not actually belong to any Buddhist order and just keep the cash for themselves. Actual monks would never seek donations in this manner. If you want to donate money, put it into the donation box in temples.'},
 {'city': 'Seoul',
  'section': 'Buy',
  'subsection': 'Fashion',
  'text': "Fashion shopping in Seoul isn't a mere industry, it's an art form. Trends often begin in University areas like Hongdae . Hongik University boasts Korea's most famous art school, thus fashion in this area is often influenced by the students' artistic sensibilities. The shops in this area feature funky, punky, boho, and vintage style. Ewha Women's University also has a big shopping area in front of its main gate, as do many of 

## Create hashid for the text 

In [7]:
import hashlib
from tqdm.auto import tqdm
import json

/mnt/d/Travel Assistant/Musafir/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def generate_document_id(doc):
    combined = f"{doc['city']}-{doc['section']}-{doc['text'][:50]}" # to be sure the data is unqiue 
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [9]:
for doc in tqdm(documents):
    doc['id'] = generate_document_id(doc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 543/543 [00:00<00:00, 539310.22it/s]


In [10]:
documents[:6]

[{'city': 'Cairo',
  'section': 'See',
  'subsection': 'Further afield',
  'text': '29.8 31.233333 1 Dahshur Pyramids . For a contrast to touristy Pyramids of Giza, head south to the oldest known pyramid, the Red Pyramid. The neglected Dahshur Pyramids are interesting and worth a visit, considering its history and the hassle-free atmosphere. Also, see the weird Bent Pyramid there and hike around the area to the Black Pyramid. The Red Pyramid has an entrance to the inside, which you can climb down. ( updated Jan 2018 )',
  'id': 'f7845786'},
 {'city': 'Cairo',
  'section': 'See',
  'subsection': nan,
  'text': '30.0405 31.2646 2 Al-Azhar Park . A landscaped garden overlooking the Citadel',
  'id': 'fca551b1'},
 {'city': 'Cairo',
  'section': 'See',
  'subsection': nan,
  'text': '30.0417 31.2483 4 Abdeen Palace . About 1 km east of Midan El-Tahrir, it was the royal residence until the Egyptian monarchy was deposed in 1952. ( updated Jan 2018 )',
  'id': '91c82200'},
 {'city': 'Cairo',
 

In [11]:
# Extract all IDs
ids = [doc["id"] for doc in documents]

# Check duplicates
duplicates = set([i for i in ids if ids.count(i) > 1])

if duplicates:
    print("Duplicated IDs found:", duplicates)
else:
    print("All IDs are unique")


All IDs are unique


In [12]:
# Save the updated json file 

with open("../data/processed_data/documents-with-ids.json", "wt") as f_out:
    json.dump(documents, f_out, indent=2)

In [13]:
!head ../data/processed_data/documents-with-ids.json

[
  {
    "city": "Cairo",
    "section": "See",
    "subsection": "Further afield",
    "text": "29.8 31.233333 1 Dahshur Pyramids . For a contrast to touristy Pyramids of Giza, head south to the oldest known pyramid, the Red Pyramid. The neglected Dahshur Pyramids are interesting and worth a visit, considering its history and the hassle-free atmosphere. Also, see the weird Bent Pyramid there and hike around the area to the Black Pyramid. The Red Pyramid has an entrance to the inside, which you can climb down. ( updated Jan 2018 )",
    "id": "f7845786"
  },
  {
    "city": "Cairo",


## Genrated the ground_truth dataste 

In [14]:
prompt_template = """
You are a traveller using our travel assistant app.
From the provided travel text, create 5 realistic questions a traveller might ask.  
- Each question must be specific and self-contained.  
- Do not copy long phrases verbatim from the text; paraphrase naturally.  
- Each question must have a short, factual answer derived only from the text.  
- Avoid generic or overly broad questions.

The record: 
City: {city},
Section: {section},
Subsection: {subsection},
Text: {text}

Provide the output in parsable JSON without using code blocks:

{{"questions": ["question1", "question2", ..., "question5"]}}
""".strip()

In [15]:
prompt = prompt_template.format(**documents[0])

In [16]:
print(prompt)

You are a traveller using our travel assistant app.
From the provided travel text, create 5 realistic questions a traveller might ask.  
- Each question must be specific and self-contained.  
- Do not copy long phrases verbatim from the text; paraphrase naturally.  
- Each question must have a short, factual answer derived only from the text.  
- Avoid generic or overly broad questions.

The record: 
City: Cairo,
Section: See,
Subsection: Further afield,
Text: 29.8 31.233333 1 Dahshur Pyramids . For a contrast to touristy Pyramids of Giza, head south to the oldest known pyramid, the Red Pyramid. The neglected Dahshur Pyramids are interesting and worth a visit, considering its history and the hassle-free atmosphere. Also, see the weird Bent Pyramid there and hike around the area to the Black Pyramid. The Red Pyramid has an entrance to the inside, which you can climb down. ( updated Jan 2018 )

Provide the output in parsable JSON without using code blocks:

{"questions": ["question1", "q

In [17]:
from mistralai import Mistral
from mistralai.models import UserMessage
import os
from dotenv import load_dotenv

In [18]:
# loads variables from .env
load_dotenv()  

True

In [19]:
api_key = os.getenv("API_KEY")

In [20]:
client = Mistral(api_key = api_key)
response = client.chat.complete(
    model= "ministral-8b-latest",
    messages=[UserMessage(content=prompt)],
)

print(response.choices[0].message.content)

{"questions": [
  "What is the name of the oldest known pyramid in Dahshur?",
  "Can I enter the Red Pyramid and climb down inside?",
  "What is the name of the pyramid with a bent shape in Dahshur?",
  "How far is the Bent Pyramid from the Red Pyramid?",
  "What is the name of the pyramid located near the Black Pyramid?"
]}


## Generate Questions


In [21]:
def generate_question(doc):
    prompt = prompt_template.format(**doc)
    client = Mistral(api_key = api_key)
    response = client.chat.complete(
        model= 'mistral-large-2411',
        messages=[UserMessage(content=prompt)],
    )    
    json_response = response.choices[0].message.content
    return json_response

In [22]:
results = {}

In [39]:
for doc in tqdm(documents):
    doc_id = doc['id']
    if doc_id in results:
        continue 
    question_raw = generate_question(doc)
    question = json.loads(question_raw)
    results[doc_id]=question['questions']

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 543/543 [00:10<00:00, 50.86it/s]


In [43]:
results['845d3951']

['What are the names of the two districts west of Jongro and Jung?',
 'Which areas in Seoul are known for their vibrant nightlife?',
 'What kind of institutions are prevalent in Seodaemun and Mapo?',
 'Can you name the two nightlife districts in the Seodaemun and Mapo area?',
 'Which district in Seoul is known for Hongdae?']

In [56]:
len(results)

543

In [57]:
# save the results

final_results = []
for doc_id, qs in results.items():
    for q in qs:
        final_results.append((doc_id, q))

In [62]:
final_results[:5]

[('f7845786', 'What is the name of the oldest known pyramid in Dahshur?'),
 ('f7845786', 'Which pyramid in Dahshur has an entrance to the inside?'),
 ('f7845786', 'What is the distinctive feature of the Bent Pyramid?'),
 ('f7845786', 'How many pyramids are mentioned to be in the Dahshur area?'),
 ('f7845786',
  'What is the atmosphere around Dahshur Pyramids like compared to Giza?')]

In [64]:
print(f"The length of the final result is: {len(final_results)}")

The length of the final result is: 2715


In [63]:
df_gt = pd.DataFrame(final_results, columns=['id', 'question'])

In [67]:
RESULT_FOLDER = "../data/result"
os.makedirs(RESULT_FOLDER, exist_ok=True)



In [68]:
df_gt.to_csv(f"{RESULT_FOLDER}/groud-truth-retrieval.csv", index=False)